In [1]:
#%pip install transformers
#!pip install pandas
#%pip install torch

import pandas as pd
import os
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import numpy as np
import re

In [ ]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
#print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
#print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

Model 1: https://huggingface.co/roberta-base
Model 2: https://huggingface.co/vinai/bertweet-base

In [ ]:
class IntersentenceEvaluator():
    

In [ ]:
choices = {'bias':1, 'unbiased':2,'unrelated':3} 
file_path = 'testing_data_inter.csv' #"drive/MyDrive/Final_templates.csv"#"drive/MyDrive/New_templates.csv"
model_name = 'distilroberta-base'
templates = pd.read_csv(file_path, sep=";")
#evaluator = IntersentenceEvaluator(templates.copy(), choices, model_name)
#evaluator.run_model_and_evaluate()